**Captsone 1: Tip analysis project from Alan Fodjo**
Through this projet, we will try to understand the factors that affect how much people tip.

First of all, we will upload our dataset

In [ ]:
import pandas as pd

df = pd.read_csv ('/content/tip.csv')
print(df)

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]


We create descriptive statistics and a column name tips_pct.

In [ ]:
#Create the column tips_pct
df['tips_pct'] = (df['tip'] / df['total_bill']) * 100

#Create descriptive statistics
avg_bill = round (df['total_bill'].mean(),2)
avg_tip = round (df['tip'].mean(),2)
avg_tip_pct = round (df['tips_pct'].mean(),2)

print (df)

     total_bill   tip     sex smoker   day    time  size   tips_pct
0         16.99  1.01  Female     No   Sun  Dinner     2   5.944673
1         10.34  1.66    Male     No   Sun  Dinner     3  16.054159
2         21.01  3.50    Male     No   Sun  Dinner     3  16.658734
3         23.68  3.31    Male     No   Sun  Dinner     2  13.978041
4         24.59  3.61  Female     No   Sun  Dinner     4  14.680765
..          ...   ...     ...    ...   ...     ...   ...        ...
239       29.03  5.92    Male     No   Sat  Dinner     3  20.392697
240       27.18  2.00  Female    Yes   Sat  Dinner     2   7.358352
241       22.67  2.00    Male    Yes   Sat  Dinner     2   8.822232
242       17.82  1.75    Male     No   Sat  Dinner     2   9.820426
243       18.78  3.00  Female     No  Thur  Dinner     2  15.974441

[244 rows x 8 columns]


**Dashboard 1: Average bill and tip**

In [ ]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import row, column

output_notebook ()
#Split the total_bill column into 20 intervals
hist_bill, edges_bill = np.histogram(df['total_bill'], bins=20)

p1 = figure(title="Distribution of invoice amounts",
            x_axis_label="Amount of the bill ($)",
            y_axis_label="Frequency",
            width=600, height=400)
p1.quad(top=hist_bill, bottom=0, left=edges_bill[:-1], right=edges_bill[1:],
        fill_color="#5cdac5", line_color="white", alpha=0.7)

#Histogramme du tip
hist_tip, edges_tip = np.histogram(df['tip'], bins=20)
p2 = figure(title="Distribution of tip amounts",
            x_axis_label= "Tips amount ($)",
            y_axis_label= "Frequency",
            width=600, height=400)
p2.quad(top=hist_tip, bottom=0, left=edges_tip[:-1], right=edges_tip[1:],
        fill_color="#6a7fd2", line_color="white", alpha=0.7)

#Dashboard layout
dashboard = row(p1,p2)

show (dashboard)

The first histogram shows that the most common invoice amounts are between $10 and $20.
The second histogram shows that the most common tips amount are between 2$ and 4$ per biil.

**Dashboard 2: Sex influence on tips**

In [ ]:
from bokeh.models import ColumnDataSource
#We create a summary table that summarizes the main statistics to be displayed.
summary = df.groupby('sex').agg(
    avg_tip = ('tip','mean'),
    avg_bill= ('total_bill','mean')
).reset_index()

#Add tip/bill ratio
summary['tip_ratio'] = summary['avg_tip'] / summary ['avg_bill']

source = ColumnDataSource (summary)
print (summary)

      sex   avg_tip   avg_bill  tip_ratio
0  Female  2.833448  18.056897   0.156918
1    Male  3.089618  20.744076   0.148940


In [ ]:
from bokeh.core.enums import Palette
from bokeh.transform import factor_cmap
p3 =  figure(
    x_range=summary['sex'].tolist(),
    title="Average tip by gender",
    y_axis_label="Average tip ($)",
    x_axis_label="Gender",
    height=350,
    width=500
)
p3.vbar (
    x='sex', top='avg_tip', width=0.4,source=source,
    fill_color= factor_cmap('sex', palette=["#e70bb3eb", "#6a7fd2"], factors=summary ['sex'])
)

p4= figure (
    x_range=summary['sex'].tolist(),
    title= 'Tip/bill ratio by gender',
    x_axis_label= 'Gender',
    y_axis_label= 'Tip/bill ratio'
)
p4.vbar (
    x='sex', top='tip_ratio', width=0.4, source = source,
    fill_color = factor_cmap ('sex', palette= ['#e70bb3eb','#6a7fd2'],
                              factors=summary['sex'])
)

show (row(p3,p4))

As stated above, Women tip more than 15% (15.7%) of their bill, while men tip less than 15% (14.8%) of their bill.

**Visula 3: Impact of smoking**
The purpose is to show if the smokers spend more or less in tips

In [ ]:
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.palettes import RdYlBu
#We create first a pivot table

pivot = df.groupby(['sex', 'smoker']).agg(avg_ratio=('tips_pct', 'mean'))
pivot = pivot.reset_index()

#Clean up the columns
pivot.columns = pivot.columns.str.strip().str.lower()
pivot_table = pivot.pivot(index='sex', columns='smoker', values ='avg_ratio' )
pivot_melted =  pivot_table.reset_index().melt(id_vars='sex', var_name='smoker', value_name='avg_ratio')

mapper = LinearColorMapper (palette=RdYlBu[6],
                            low=pivot_melted.avg_ratio.min(),
                            high=pivot_melted.avg_ratio.max())

source1 =ColumnDataSource(pivot_melted)

In [ ]:
from bokeh.transform import transform
p5 = figure (
    title = 'Tip ratio/total bill by gender and smoking status',
    x_range = sorted (pivot_melted['smoker'].unique()),
    y_range =  list (reversed(sorted(pivot_melted['sex'].unique()))),
    width=500, height=400,
    toolbar_location=None
)

p5.rect(x='smoker', y='sex', width = 1, height=1,
        source=source1,
        line_color=None,
        fill_color= transform('avg_ratio', mapper)
        )

color_bar = ColorBar(color_mapper=mapper, title="Tip moyen", location=(0,0))
p5.add_layout (color_bar, 'right')

show (p5)

In [ ]:
from bokeh.models import Label
#Create an array to group tips and average bill by smoker/nonn smokers
smoke = df.groupby('smoker').agg(
    avg_tip = ('tip', 'mean'),
    avg_bill = ('total_bill', 'mean')).reset_index ()

#We calculate the delta between avg tips
delta_tip = (smoke['avg_tip'][1] - smoke['avg_tip'][0]) / smoke['avg_tip'][0] * 100
delta_text = f"Δ = {delta_tip:.2f}% "
source2=ColumnDataSource(smoke)

p6 = figure (
    x_range=smoke['smoker'],
    title='Average tip by category',
    x_axis_label= 'Smoker (Yes/No)',
    y_axis_label= 'Average tip $',
    height= 350,
    width=500
)
p6.vbar (
    x = 'smoker', top = 'avg_tip', width = 0.4, source=source2,
    fill_color= factor_cmap('smoker', palette=['#000000',"#C01414FE" ], factors= smoke['smoker'])
)
annotation = Label(x=1, y=3.03, text=delta_text,
                   text_color="darkred", text_font_size="11pt")
p6.add_layout(annotation)

show (p6)

The graph shows that smoker give more tips than non smoker (0,5% more)

**Dashboard 4:Infleunce of the service time**
we want to compare the total bill and tips by lunch and dinner time

In [ ]:
time = df.groupby('time').agg(total = ('total_bill', 'sum'),
                               avg_tip = ('tip','mean'),
                               avg_bill= ('total_bill','mean'))

time = time.reset_index()
source3 = ColumnDataSource (time)

colors = ['#000000',"#7A696984"]
#P7 is the total tips by time
p7 = figure (
    title= 'Meal time with the highest tip',
    x_range=time['time'].to_list(),
    y_range= (0,4000),
    x_axis_label= 'Meal time',
    y_axis_label= 'tips total',
    height= 300, width= 800
)
cmap = factor_cmap ('time', palette=colors, factors=time['time'].to_list())
p7.vbar (x='time', top = 'total', source=source3,
         width = 0.7, legend_field='time', fill_color=cmap)

p7.legend.orientation = 'horizontal'
p7.legend.location = 'top_center'

#p8 is average tip by time
p8 = figure (
    x_range=time['time'].tolist(),
    title="Average tip per hour of service",
    y_axis_label="Average tip ($)",
    x_axis_label="Meal time",
    height=350,
    width=500
)
p8.vbar (
    x='time', top='avg_tip', width=0.4,source=source3,
    fill_color= factor_cmap('time', palette=["#000000", "#7A696984"],
    factors=time['time'].tolist())
)

#p9 is average bill by time
p9 = figure (
    title = "Average bill per hour of service",
    x_range = time['time'].tolist(),
    x_axis_label= 'Meal time',
    y_axis_label= 'Average bill in $',
    height=350,
    width=500
)
p9.vbar (
    x='time', top='avg_bill', width=0.4,source=source3,
    fill_color= factor_cmap('time', palette=["#000000", "#7A696984"],
    factors=time['time'].tolist())
)

show (row(p7, p8, p9))

**Dashboard 5: Bill per day and service time**
With this dashboard, we will see which days and times are the busiest.

In [ ]:
from bokeh.models import FactorRange

#we group 'day' and 'time' to obtain the total bills
grouped = df.groupby(['day', 'time']).agg(total=('total_bill','sum')).reset_index()

factors = [(row['day'], row['time']) for _, row in grouped.iterrows()]

source4 = ColumnDataSource(data=dict(factors=factors, total=grouped['total']))

#color palette to differentiate lunch/dinner
palette1 = ['#5cdac5', '#6a7fd2']

#creating the chart
p10 = figure (
    title= 'Total sales by day and meal time',
    x_range=FactorRange (*factors),
    height=400,
    width=800,
    x_axis_label='Day and time',
    y_axis_label= "Total sales"
)

p10.vbar (
    x='factors',
    top='total',
    width=0.7,
    source=source4,
    fill_color=factor_cmap('factors', palette=palette1,
                           factors=[('Lunch'),('Dinner')], start = 1, end = 2)
)

#To make your chart more readable and attractive.
show (p10)